In [1]:
from api import API
import pandas as pd

In [2]:
api = API()

## Load / Merge Data

In [3]:
d = api.buildings()
bldings = list(d.values())

In [4]:
#  Jons stuff

# import sys
# sys.path.insert(0, '..')

# from marthas_dashboard.views import (
#     get_room_comparison_results,
#     get_room_names_from_point_names)

# searches = dict()
# searches['building'] = '4'
# searches['date'] = '2017-08-18'
# searches['timestamp'] = '00:00:00'

# search_results = get_room_comparison_results(searches)
# search_results.head()

# sr2 = get_room_names_from_point_names(searches, vals)
# sr2.head()

In [5]:
bldings_with_rooms = ['Hulings']

rm = bldings_with_rooms[0]
print(rm)
bld = api.building(rm).id
timestamp = '2017-08-18 00:45:00'

# Load some data
rooms = api.building_rooms(bld)
points = api.building_points(bld)
vals = api.building_values_at_time(bld, timestamp)

Hulings


In [6]:
# display(rooms.head())
# display(points.head())
# display(vals.head())

In [7]:
# Remove room 6, the scary dummy room
rooms = rooms[rooms['name'].str.find('_Dummy_') < 0]

In [8]:
rooms.head()

,buildingid,id,name
0,4,4,Hulings_Room
1,4,5,218
3,4,7,300
4,4,8,302
5,4,9,310


In [9]:
# Merge df together
df = pd.merge(
    points, rooms,
    left_on='roomid', right_on='id',
    suffixes=('_point', '_room'))

df= pd.merge(df, vals, left_on='id_point', right_on='pointid',)

In [11]:
df.head(n=10)

,description,equipmentid,id_point,name_point,pointsourceid,pointtypeid,roomid,buildingid,id_room,name_room,...,id,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time
0,112 RH VALVE,NaN,572,HU.R112.RHV,2,1,18,4,18,112,...,1,floatPSI2,572,HU.R112.RHV,2017-08-18 00:45:00,3.00,float,PSI,2017-08-18,00:45
1,112 TEMP,NaN,573,HU.R112.RM,2,13,18,4,18,112,...,13,floatDEG F2,573,HU.R112.RM,2017-08-18 00:45:00,70.92,float,DEG F,2017-08-18,00:45
2,112 TEMP,NaN,574,HU.R112.RMT,2,13,18,4,18,112,...,13,floatDEG F2,574,HU.R112.RMT,2017-08-18 00:45:00,70.92,float,DEG F,2017-08-18,00:45
3,112 SUP CFM,NaN,575,HU.R112.SACFM,2,699,18,4,18,112,...,699,floatC CFM2,575,HU.R112.SACFM,2017-08-18 00:45:00,0.00,float,C CFM,2017-08-18,00:45
4,112 TOT SUP,NaN,576,HU.R112.TSUP,2,47,18,4,18,112,...,47,floatCFM2,576,HU.R112.TSUP,2017-08-18 00:45:00,0.00,float,CFM,2017-08-18,00:45
5,121 EXH CFM,NaN,587,HU.R121.ECFM,2,47,19,4,19,121,...,47,floatCFM2,587,HU.R121.ECFM,2017-08-18 00:45:00,127.87,float,CFM,2017-08-18,00:45
6,121 EXH CFM,NaN,588,HU.R121.EXCFM,2,699,19,4,19,121,...,699,floatC CFM2,588,HU.R121.EXCFM,2017-08-18 00:45:00,1.28,float,C CFM,2017-08-18,00:45
7,121 RH VALVE,NaN,589,HU.R121.RHV,2,1,19,4,19,121,...,1,floatPSI2,589,HU.R121.RHV,2017-08-18 00:45:00,9.00,float,PSI,2017-08-18,00:45
8,121 TEMP,NaN,590,HU.R121.RM,2,13,19,4,19,121,...,13,floatDEG F2,590,HU.R121.RM,2017-08-18 00:45:00,70.61,float,DEG F,2017-08-18,00:45
9,121 TEMP,NaN,591,HU.R121.RMT,2,13,19,4,19,121,...,13,floatDEG F2,591,HU.R121.RMT,2017-08-18 00:45:00,70.61,float,DEG F,2017-08-18,00:45


## Attempt to tag specific points

In [12]:
df['tag'] = 'none'

# Tag 'valve' based on description
df_valve = df[df['description'].str.find('VALVE') > 0]
df.loc[df_valve.index, 'tag'] = 'valve'

# Tag 'temp' based on point name
df_temp = df[df['name_point'].str.find('.RM') > 0]
df.loc[df_temp.index, 'tag'] = 'temp1 (RM)'

df_temp = df[df['name_point'].str.find('.RMT') > 0]
df.loc[df_temp.index, 'tag'] = 'temp2 (RMT)'

In [13]:
# View DF where tag is not "none
view_df = df.query('tag != "none"')

# Pivot 
view_df = (view_df.pivot(index='name_room', columns='tag',
                        values='pointvalue')
           .reset_index().rename_axis(None, axis=1))

view_df

,name_room,temp1 (RM),temp2 (RMT),valve
0,102,73.39,73.39,12.0
1,112,70.92,70.92,3.0
2,121,70.61,70.61,9.0
3,202,69.60,71.01,15.0
4,203,67.65,67.65,9.0
5,206,69.10,69.10,9.0
6,207,76.93,76.93,9.0
7,212,72.18,72.18,3.0
8,213,71.60,71.60,9.0
9,215,69.11,69.11,9.0


In [14]:
for idx, row in view_df.iterrows():
    print(row.name_room)

102
112
121
202
203
206
207
212
213
215


In [13]:
view_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>name_room</th>\n      <th>temp1 (RM)</th>\n      <th>temp2 (RMT)</th>\n      <th>valve</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>102</td>\n      <td>73.39</td>\n      <td>73.39</td>\n      <td>12.0</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>112</td>\n      <td>70.92</td>\n      <td>70.92</td>\n      <td>3.0</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>121</td>\n      <td>70.61</td>\n      <td>70.61</td>\n      <td>9.0</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>202</td>\n      <td>69.60</td>\n      <td>71.01</td>\n      <td>15.0</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>203</td>\n      <td>67.65</td>\n      <td>67.65</td>\n      <td>9.0</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>206</td>\n      <td>69.10</td>\n      <td>69.10</td>\n      <td>9.0</td>\n    </tr>\n    <tr>\n      <th

In [ ]:
def get_rooms_points(buildings):
    """maps building ids to their points and rooms,
    ie {4:{'rooms':{5}}}"""
    result = {}
    for building_id, name in buildings.items():
        building_data = {
            'rooms': map_rooms(api.building_rooms(building_id)),
            'points': map_points(api.building_points(building_id))
        }
        result[building_id] = building_data
    return result

def map_points(points):
    """Take a pandas df returned from building_points
    and turn it into something easier to use on the frontend
    """
    results = {}
    for index, row in points.iterrows():
        results[row['id']] = row['name'] + '- ' + row['description']
    return results


def map_rooms(rooms):
    """Similar to the map_points function but for rooms"""
    results = {}
    for index, row in rooms.iterrows():
        results[row['id']] = row['name'].replace("_", " ")
    return results